# Evaluate MENT-Flow model

In [ ]:
import os
import pickle
import sys

import matplotlib.pyplot as plt
import numpy as np
import proplot as pplt
import psdist as ps
import psdist.visualization as psv
import scipy.ndimage
import torch
import zuko
from ipywidgets import interact
from ipywidgets import widgets

import mentflow as mf
from mentflow.train.plot import set_proplot_rc
from mentflow.utils import grab
from mentflow.utils import unravel

sys.path.append("../..")
from experiments.load import load_mentflow_run
from experiments.rec_nd_2d.setup import make_distribution

In [ ]:
set_proplot_rc()

## Settings

In [ ]:
device = "cpu"
precision = torch.float32


def send(x):
    return x.type(precision).to(device)

## Load data

In [ ]:
script_name = "train_flow"
timestamp = 240413163225
data_dir = f"./outputs/{script_name}/{timestamp}/"

In [ ]:
run = load_mentflow_run(data_dir, device=device)

cfg = run["config"]
history = run["history"]
model = run["model"]
checkpoints = run["checkpoints"]

In [ ]:
distribution = make_distribution(cfg)

## Scalar history

In [ ]:
keys_sorted = sorted(list(history.keys()))
imax = len(history[keys_sorted[0]])


@interact(
    key=widgets.Dropdown(options=keys_sorted, value="D_norm"),
    irange=widgets.IntRangeSlider(min=0, max=imax, value=(0, imax)),
    log=False,
)
def update(key: str, irange: int, log: bool):
    vals = history[key]
    avgs = mf.utils.exp_avg(vals, momentum=0.95)

    fig, ax = pplt.subplots()
    ax.plot(vals[irange[0] : irange[1]], color="gray")
    ax.plot(avgs[irange[0] : irange[1]], color="black", lw=1.0)
    if log:
        ax.format(yscale="log", yformatter="log")
    ax.format(xlabel="Iteration (global)", ylabel=key)
    plt.show()

## Evaluation

### Radial CDF

In [ ]:
@interact(
    index=widgets.IntSlider(
        min=0,
        max=(len(checkpoints) - 1),
        value=(len(checkpoints) - 1),
    ),
    n_samples=widgets.FloatLogSlider(min=2, max=6, value=1.00e05),
    rmax=widgets.FloatSlider(min=0.0, max=6.0, value=3.0),
    bins=widgets.IntSlider(min=4, max=150, value=50),
    kind=["step", "line"],
)
def update(index, n_samples, rmax, bins, kind):
    model.load(checkpoints[index]["path"], device)

    with torch.no_grad():
        n_samples = int(n_samples)
        x1 = grab(distribution.sample(n_samples))
        x2 = grab(model.sample(n_samples))

        fig, ax = mf.train.plot.plot_dist_radial_cdf(
            x1, x2, rmax=rmax, bins=bins, kind=kind, lw=1.5, colors=["red4", "black"]
        )

        for name in ["kv", "gaussian"]:
            x = mf.distributions.get_distribution(name=name, noise=0.0).sample(n_samples)
            x = grab(x)
            hist, edges = ps.points.radial_histogram(x, bins=75, limits=(0.0, rmax))
            cdf = np.cumsum(hist)
            cdf = cdf / cdf[-1]
            psv.plot_profile(
                cdf,
                edges=edges,
                ax=ax,
                kind="line",
                color="black",
                alpha=0.1,
                ls="--",
                zorder=0,
                lw=1.5,
            )
        ax.format(xlabel="Radius", ylabel="CDF")
        pplt.show()

## 2D projections

In [ ]:
@interact(
    dim1=widgets.Dropdown(options=range(cfg.ndim), value=0),
    dim2=widgets.Dropdown(options=range(cfg.ndim), value=1),
    index=widgets.IntSlider(
        min=0,
        max=(len(checkpoints) - 1),
        value=(len(checkpoints) - 1),
    ),
    n=widgets.FloatLogSlider(min=2.0, max=6.0, value=1.00e05),
    xmax=widgets.FloatSlider(min=0.0, max=6.0, value=3.0),
    bins=widgets.IntSlider(min=4, max=150, value=85),
)
def update(dim1, dim2, index, n, xmax, bins):
    if dim1 == dim2:
        return
        
    model.load(checkpoints[index]["path"], device)

    with torch.no_grad():
        x1 = grab(distribution.sample(int(n)))
        x2 = grab(model.sample(int(n)))

        fig, axs = pplt.subplots(ncols=2, xspineloc="neither", yspineloc="neither")
        for x, ax in zip([x1, x2], axs):
            psv.points.plot2d(
                x[:, (dim1, dim2)],
                bins=bins,
                limits=(2 * [(-xmax, xmax)]),
                ax=ax,
                mask=False,
            )
        pplt.show()

### Slices 

In [ ]:
n = 100_000
model.load(checkpoints[-1]["path"], device)
with torch.no_grad():
    x1 = grab(distribution.sample(int(n)))
    x2 = grab(model.sample(int(n)))

psv.points.proj2d_interactive_slice(
    data=[[x1], [x2]],
    slice_res=7,
    plot_res=50,
    options=dict(mask=True),
    cmap="viridis",
    autolim_kws=dict(sigma=3.5)
)

### Spherical slices 

In [ ]:
n = 100_000
model.load(checkpoints[-1]["path"], device)
with torch.no_grad():
    x1 = grab(distribution.sample(int(n)))
    x2 = grab(model.sample(int(n)))

@interact(rmax=(0.0, 6.0), log=False)
def update(rmax, log):
    fig, axs = pplt.subplots(ncols=2)
    limits = 2 * [(-3.5, 3.5)]
    for ax, x in zip(axs, [x1, x2]):
        x_slice = ps.points.slice_sphere(
            x, axis=range(2, x.shape[1]), rmin=0.0, rmax=rmax
        )
        psv.points.plot2d(
            x_slice[:, :2],
            ax=ax,
            bins=50,
            limits=limits,
            cmap="viridis",
            mask=False,
            norm=("log" if log else None),
            offset=1.0,
            rms_ellipse=True,
            rms_ellipse_kws=dict(level=2.0, color="white"),
        )
    axs.format(xlim=limits[0], ylim=limits[1])

### Corner plot

In [ ]:
@interact(
    index=widgets.IntSlider(
        min=0,
        max=(len(checkpoints) - 1),
        value=(len(checkpoints) - 1),
    ),
    n=widgets.FloatLogSlider(min=2, max=6, value=1.00e05, continuous_update=False),
    xmax=widgets.FloatSlider(min=0.0, max=6.0, value=3.0),
    bins=widgets.IntSlider(min=4, max=150, value=85),
)
def update(index, n, xmax, bins):
    model.load(checkpoints[index]["path"], device)

    with torch.no_grad():
        x1 = grab(distribution.sample(int(n)))
        x2 = grab(model.sample(int(n)))

        mask = True

        grid = psv.CornerGrid(d=x1.shape[1], corner=False)
        limits = [(-xmax, xmax)] * x1.shape[1]
        grid.plot_points(
            x2,
            lower=True,
            upper=False,
            mask=mask,
            bins=bins,
            limits=limits,
            cmap="blues",
            diag_kws=dict(color="blue7", lw=1.5),
        )
        grid.plot_points(
            x1,
            upper=True,
            lower=False,
            mask=mask,
            bins=bins,
            limits=limits,
            cmap="reds",
            diag_kws=dict(color="red7", lw=1.5),
        )
        pplt.show()

### Simulated measurements

In [ ]:
@interact(
    index=widgets.IntSlider(
        min=0,
        max=(len(checkpoints) - 1),
        value=(len(checkpoints) - 1),
    ),
    n=widgets.FloatLogSlider(min=2, max=6, value=50000),
    log_ymin=widgets.FloatSlider(min=-5.0, max=-1.0, value=-3.0),
    maxcols=widgets.IntSlider(min=4, max=12, value=7),
    figwidthscale=widgets.FloatSlider(min=0.5, max=2.5, value=1.25),
)
def update(index, n, log_ymin, maxcols, figwidthscale, kde=False, log=False):
    model.load(checkpoints[index]["path"], device=device)

    with torch.no_grad():
        # Generate samples from the model.
        x = model.sample(int(n))
        x = send(x)

        # Compute projections.
        for diagnostic in unravel(model.diagnostics):
            diagnostic.kde = kde

        predictions = mf.simulate.forward(x, model.transforms, model.diagnostics)

        y_meas = [grab(meas) for meas in unravel(model.measurements)]
        y_pred = [grab(pred) for pred in unravel(predictions)]
        coords = [[grab(0.5 * (e[:-1] + e[1:])) for e in diag.edges] for diag in unravel(model.diagnostics)]

        # Setup figure.
        ncols = min(len(y_meas), maxcols)
        nrows = 2 * int(np.ceil(len(y_meas) / ncols))

        fig, axs = pplt.subplots(
            ncols=ncols,
            nrows=nrows,
            figwidth=min(figwidthscale * ncols, 10.0),
            xspineloc="neither",
            yspineloc="neither",
            space=0.0,
        )

        # Plot truth in first row, predictions in second row, etc.
        kws = dict(
            norm=("log" if log else None),
        )
        i = 0
        for row in range(0, nrows, 2):
            for col in range(ncols):
                if i < len(y_meas):
                    ax_index = row * ncols + col
                    scale = np.max(y_meas[i])
                    psv.image.plot2d(
                        y_meas[i] / scale, coords=coords[i], ax=axs[ax_index], **kws
                    )
                    psv.image.plot2d(
                        y_pred[i] / scale,
                        coords=coords[i],
                        ax=axs[ax_index + ncols],
                        **kws
                    )
                i += 1

        plt.show()

### Grid warp 

In [ ]:
@interact(
    index=widgets.IntSlider(
        min=0,
        max=(len(checkpoints) - 1),
        value=(len(checkpoints) - 1),
    ),
    xmax=widgets.FloatSlider(min=0.0, max=6.0, value=3.0),
    n_lines=widgets.IntSlider(min=0.0, max=200, value=100),
)
def update(index, xmax, n_lines):
    model.load(checkpoints[index]["path"], device=device)

    with torch.no_grad():
        fig, axs = pplt.subplots(ncols=2)

        res = 150
        grid = np.linspace(-3.5 * xmax, 3.5 * xmax, n_lines)
        line_points = []
        for i in range(len(grid)):
            x = np.full(res, grid[i])
            y = np.linspace(-xmax, xmax, res)
            line_points.append(np.vstack([x, y]).T)
            line_points.append(np.vstack([y, x]).T)

        kws = dict(color="black", lw=0.6, alpha=0.25)

        for z in line_points:
            axs[0].plot(z[:, 0], z[:, 1], **kws)
            z = np.hstack([z, np.zeros((z.shape[0], cfg.ndim - z.shape[1]))])
            z = send(torch.from_numpy(z))
            x = grab(model.generator.forward(z))
            for ax in axs[1:]:
                ax.plot(x[:, 0], x[:, 1], **kws)

        z = model.generator.sample_base(1000)
        x = model.generator.forward(z)
        z = grab(z)
        x = grab(x)
        kws = dict(c="black", zorder=999, s=1)
        axs[0].scatter(z[:, 0], z[:, 1], **kws)
        axs[1].scatter(x[:, 0], x[:, 1], **kws)
        axs.format(xlim=(-xmax, xmax), ylim=(-xmax, xmax))
        plt.show()

### Flow trajectory

In [ ]:
@interact(
    index=widgets.IntSlider(
        min=0,
        max=(len(checkpoints) - 1),
        value=(len(checkpoints) - 1),
    ),
    n=widgets.FloatLogSlider(min=2, max=6, value=1.00e05),
    xmax=widgets.FloatSlider(min=0.0, max=6.0, value=3.0),
    bins=widgets.IntSlider(min=4, max=150, value=75),
)
def update(index, n, xmax, bins):
    model.load(checkpoints[index]["path"], device)

    with torch.no_grad():
        z = model.generator.sample_base(int(n))
        try:
            xt = model.generator.forward_steps(z)
        except:
            print("`model.gen` does not have `forward_steps` method.")

        fig, axs = pplt.subplots(
            figheight=2.0,
            ncols=len(xt),
            space=None,
            xticks=[],
            yticks=[],
            xspineloc="neither",
            yspineloc="neither",
        )
        limits = 2 * [(-xmax, xmax)]
        for ax, x in zip(axs, xt):
            x = grab(x)
            ax.hist2d(x[:, 0], x[:, 1], bins=bins, range=limits)
        axs.format(xlim=limits[0], ylim=limits[1])
        plt.show()